In [1]:
# bus_analyser.py

import csv, re

In [4]:
test_filepath = "../saved/65 - na Tulaka.tsv"

In [6]:
with open(test_filepath, encoding="1251") as f:
    data = list(csv.reader(f, delimiter="\t"))

In [7]:
len(data)

6630

In [9]:
header = data[0]

In [10]:
for row in data:
    if row == header:
        print(row[0])

Дата
Дата
Дата
Дата
Дата
Дата
Дата
Дата
Дата
Дата
